In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from IPython.display import display
from matplotlib.ticker import MaxNLocator
from matplotlib import ticker
import math
from scipy.interpolate import make_interp_spline
from matplotlib.pyplot import cm
import numpy.random as npr
from spyrmsd import rmsd
import os
from os import listdir
from os.path import isfile, join
import glob
import natsort 
import shutil
from pathlib import Path
from os.path import exists
import sys
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_colwidth', None)

In [171]:
def sdf_files_manager():
    working_folder = input('provide a foldername: ')
    path2 = os.chdir(working_folder)
    folders = [f for f in listdir(path2)]
    if 'merged_sdfs' in folders:
        print('output folder is already present! Exiting the script')
        sys.exit()
        
    names_list = []
    names_sdf_list = []
    sorted_names_sdf_nondup = []
    dups = []
    index = [i for i in range(1, len(folders))]
    count = 0

    for root, dirs, files in os.walk(working_folder, topdown=False):
        for name in files:
            if name != '.DS_Store' and '.sdf' in name and len(name)>9:
                if name in names_list:
                    os.rename(root+'/'+name, root+'/'+name+'_'+str(index[count])+'.sdf')
                    names_list.append(name)
                else: 
                    names_list.append(name)
        count+=1
    names_list.sort(key=lambda x: x.split(("-"))[1])

    directory_name = 'merged_sdfs'
    folder_merged_dest = os.path.join(working_folder, directory_name)
    if directory_name not in folders:
        os.mkdir(folder_merged_dest)
    else:
        print('output folder already exists! Exiting the script')
        sys.exit()
        
    count_2 = 0
    for i in folders[1:]:
        for root, dirs, filenames in os.walk(i):
            for file in filenames:
                if file in names_list:
                    shutil.copy(working_folder+'/'+root+'/'+file, folder_merged_dest)

    for root, dirs, filenames in os.walk(folder_merged_dest):
        files_to_transfer = natsort.natsorted(filenames)
    if '.txt' in ''.join(files_to_transfer[1:]):
        print('.txt output is already present! will write a text file!')
        sys.exit()
    else:
        files_to_transfer.sort(key=lambda x: x.split(("-"))[1]) #'.DS store creates itself => [1:]'
    with open('output_sdfs.txt', 'w') as f:
        for i in files_to_transfer:
            f.write("%s\n" % i)
    return folder_merged_dest
processing_folder = sdf_files_manager()

def RMSD_calculation(folder):
    path = os.chdir(folder)
    onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f)) if '.txt' not in f if '.DS_Store' not in f
                ]
    onlyfiles.sort(key=lambda x: x.split(("-"))[1])
    ref_model = onlyfiles[3]
    RMSD_list = []
    scores_list = []
    for i in onlyfiles:
        if len(i) > 30:
            scores_list.append(i[-15:-10:1])
        else:
            scores_list.append(i[-9:-4:1])
#     print(len(scores_list))
    for i in range(len(onlyfiles)):
        test_model = onlyfiles[i]
        output_rmsd = !python3 -m spyrmsd -m {ref_model} {test_model}
        RMSD_list.append(output_rmsd)
        flat_RMSD_list = [i for sublist in RMSD_list for i in sublist]
        
    return scores_list, flat_RMSD_list
scores, rmsd_list = RMSD_calculation(processing_folder)

def score_RMSD_plot(scores, rmsd): 
    plot_df = pd.DataFrame({'rmsd':rmsd_list, 
                          'score':scores})
    plot_df['rmsd'] = plot_df['rmsd'].astype(float)
    plot_df['score'] = plot_df['score'].astype(float)
    scatter = plot_df.plot(x='rmsd', y='score', kind='scatter')
    ax = scatter.axes
    ax.invert_yaxis()
    ax.set_xlabel('Ligand pose RMSD [$\AA$]', fontsize = 14)
    ax.set_ylabel('Confidence score', fontsize = 14)
    ax.set_title('DiffDock #1177 top4_best', fontsize = 14)
    savefig_input = input('save the plot? y/n: ')
    if savefig_input == 'y': 
        plt.savefig(str(processing_folder)+'/'+'DiffDock #3862 top4', 
               dpi = 300, bbox_inches='tight')
    else:
        None
    plt.show()
    
score_RMSD_plot(scores, rmsd_list)